In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

🌿 Business & Research Overview

The luxury market is evolving. Conscious consumers increasingly expect brands to uphold sustainability, transparency, and ethical responsibility — transforming sustainability from a “nice-to-have” into a decisive driver of brand equity and purchase intent.
This notebook explores how consumer psychology and data science intersect to explain and predict sustainable luxury purchase behavior. Using survey data from 500 respondents, we combine behavioral insights with predictive modeling to uncover what truly influences purchase intention in the sustainable luxury domain.

🎯 Project Objective

This analysis aims to:
1.	Model consumer purchase intention toward sustainable luxury products.
2.	Identify the most influential psychological and attitudinal factors driving eco-conscious behavior.
3.	Explain model outputs through interpretable AI techniques (SHAP) to reveal why consumers choose sustainably.
  
💼 Business Relevance

For brand strategists, marketing analytics teams, and business leaders, this notebook provides a replicable framework for:
•	Quantifying how much sustainability perception impacts buying decisions.
•	Detecting high-value consumer segments based on eco-attitudes and motivation scores.
•	Designing data-informed marketing campaigns that align sustainability with brand desirability.
The findings can directly inform product positioning, communication strategy, and CSR alignment for luxury houses seeking growth through responsibility.

🧠 Technical Approach

We evaluate several machine learning models — including Linear Regression, Random Forest, XGBoost, and Optuna-tuned ensembles — to predict consumer responses.
Model transparency is ensured through:
•	Feature importance visualizations (Random Forest & XGBoost)
•	SHAP explainability to trace feature contributions at both global and individual levels
All models are wrapped in scikit-learn pipelines with robust preprocessing, ensuring clean, reproducible experimentation.

📈 Key Insights

•	Attitudinal items (ATT1–ATT5) are dominant predictors of sustainability preference.
•	Personality traits contribute marginally but help segment nuanced consumer profiles.
•	SHAP visualizations translate complex models into actionable business understanding — showing why certain traits or attitudes lead to purchase intent.

🌍 Why This Matters

As sustainability transitions from trend to expectation, data-driven empathy becomes the new competitive edge.
This notebook demonstrates how marketing science + machine learning can quantify the emotional and cognitive dimensions of sustainability — empowering decision-makers to build brands that are not only luxurious, but also responsible and future-ready.


In [ ]:
# ============================
# Sustainable Luxury: Full ML Pipeline
# ============================

# ---- Imports
import os, math, sys, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    classification_report, accuracy_score, f1_score
)
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.exceptions import UndefinedMetricWarning
from scipy.sparse import issparse
import joblib

# Notebook niceties
pd.set_option("display.max_columns", 120)

# If in a notebook, enable inline plotting (safe-guarded)
try:
    get_ipython
    # This IPython magic is fine in Kaggle notebooks
    # If you convert to .py, remove the next line.
    get_ipython().run_line_magic("matplotlib", "inline")
except Exception:
    pass

# ---- Silence specific sklearn warning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# ============================
# 1) Load Excel robustly
# ============================
candidate_paths = [
    "/kaggle/working/Sustainable_Luxury_Survey_500.xlsx",
    "/kaggle/working/sustainable_luxury_survey.xlsx",
    "/kaggle/working/Sustainable_Luxury_Survey.xlsx",
]

# Also scan /kaggle/input for Excel files if you added a Kaggle Dataset
input_dir = "/kaggle/input"
if os.path.exists(input_dir):
    for root, _, files in os.walk(input_dir):
        for f in files:
            if f.lower().endswith((".xlsx", ".xls")):
                candidate_paths.append(os.path.join(root, f))

print("Candidate paths:", candidate_paths)

df = None
for p in candidate_paths:
    if os.path.exists(p):
        try:
            df = pd.read_excel(p)
            print(f"Loaded: {p} shape={df.shape}")
            break
        except Exception as e:
            print("Tried", p, "->", e)

if df is None:
    raise FileNotFoundError(
        "No Excel found. Upload your file via the left sidebar (📁 Upload) and re-run."
    )

# ============================
# 2) Normalize column names
# ============================
df = df.rename(columns=lambda c: str(c).strip().replace(" ", "_").replace("-", "_").lower())
print("Columns:", df.columns.tolist())

# ============================
# 3) Pick targets by common aliases
# ============================
def pick_first_existing(columns, candidates):
    for c in candidates:
        if c in columns:
            return c
    return None

sust_candidates = [
    "sustainability_attitude", "sustainability_attitude_score",
    "sustainability_preference", "sustainability_preference_score",
    "attitude_toward_sustainability", "attitude_score"
]
purchase_candidates = [
    "purchase_intent", "purchase_intention", "purchase_intention_score",
    "purchase_intent_score"
]

sust_target = pick_first_existing(df.columns, sust_candidates)
purchase_target = pick_first_existing(df.columns, purchase_candidates)

if not sust_target or not purchase_target:
    raise KeyError(
        f"Could not find sustainability and/or purchase targets.\n"
        f"Columns present: {df.columns.tolist()}\n"
        f"Expected one of {sust_candidates} and one of {purchase_candidates}."
    )

print(f"Targets -> sustainability: {sust_target} | purchase: {purchase_target}")

# ============================
# 4) Build features / targets
# ============================
id_like = [c for c in df.columns if c in ["id", "respondent_id"]]
X = df.drop(columns=[sust_target, purchase_target] + id_like, errors="ignore")

y_map = {
    "sustainability": df[sust_target],
    "purchase": df[purchase_target],
}

num_cols = X.select_dtypes(include=["number"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
print("Numeric features:", num_cols[:10], f"... ({len(num_cols)} total)")
print("Categorical features:", cat_cols)

# ============================
# 5) Preprocessing + models
# ============================
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    # with_mean=False keeps it sparse-safe when combined with OneHot
    ("scaler", StandardScaler(with_mean=False)),
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols),
    ],
    remainder="drop",
)

reg_model = RandomForestRegressor(
    n_estimators=400, random_state=42, n_jobs=-1
)

clf_model = RandomForestClassifier(
    n_estimators=400, random_state=42, n_jobs=-1
)

# ============================
# 6) Regression for both targets
# ============================
def rmse(y_true, y_pred):
    return math.sqrt(mean_squared_error(y_true, y_pred))

print("\n=== REGRESSION RESULTS ===")
results_reg = {}

for name, y in y_map.items():
    if not np.issubdtype(y.dtype, np.number):
        print(f"[{name}] target is not numeric. Skipping regression for this target.")
        continue

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    reg_pipe = Pipeline(steps=[("prep", preprocess), ("model", reg_model)])
    reg_pipe.fit(X_train, y_train)

    preds = reg_pipe.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    score_rmse = rmse(y_test, preds)

    print(f"[{name}] MAE={mae:.3f} | RMSE={score_rmse:.3f} | R^2={r2:.3f}")

    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(reg_pipe, X, y, cv=cv, scoring="r2", n_jobs=-1)
    print(f"[{name}] 5-fold CV R^2: {np.round(cv_scores,3)} | mean={cv_scores.mean():.3f}")

    results_reg[name] = dict(MAE=mae, RMSE=score_rmse, R2=r2, CV_R2_mean=cv_scores.mean())

# ============================
# 7) Optional classification: bin targets into Low/Med/High
# ============================
def bin_target(series, labels=["Low", "Med", "High"]):
    """
    Use quantile bins so classes are more balanced and
    less likely to be empty -> avoids undefined metrics.
    """
    return pd.qcut(series, q=3, labels=labels, duplicates="drop")

print("\n=== CLASSIFICATION RESULTS (Low/Med/High) ===")
results_clf = {}

for name, y_raw in y_map.items():
    if not np.issubdtype(y_raw.dtype, np.number):
        print(f"[{name}] target is not numeric. Skipping classification.")
        continue

    # Make categorical with likely non-empty bins
    y_cat = bin_target(y_raw).astype("category")
    y_cat = y_cat.cat.remove_unused_categories()

    # Train/test split (stratify preferred; fallback handled)
    try:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y_cat, test_size=0.2, random_state=42, stratify=y_cat
        )
    except ValueError:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y_cat, test_size=0.2, random_state=42
        )

    clf_pipe = Pipeline(steps=[("prep", preprocess), ("model", clf_model)])
    clf_pipe.fit(X_train, y_train)
    preds = clf_pipe.predict(X_test)

    # Ensure consistent label order for reports
    labels_order = list(y_test.cat.categories) if hasattr(y_test, "cat") else sorted(pd.unique(y_test))

    acc = accuracy_score(y_test, preds)
    f1w = f1_score(y_test, preds, average="weighted", zero_division=0)
    print(f"[{name}] Acc={acc:.3f} | F1_w={f1w:.3f}")

    print(
        f"\nClassification report ({name}):\n",
        classification_report(
            y_test,
            preds,
            labels=labels_order,
            zero_division=0   # <- prevents UndefinedMetricWarning
        )
    )

    results_clf[name] = dict(Accuracy=acc, F1_weighted=f1w)

# ============================
# 8) Feature importance (approx)
# ============================
print("\n=== FEATURE IMPORTANCES (approximate) ===")

# Fit preprocess on ALL X to extract stable feature names
preprocess_fitted = preprocess.fit(X)

feature_names_num = num_cols
feature_names_cat = []
if len(cat_cols) > 0:
    feature_names_cat = preprocess_fitted.named_transformers_["cat"]["onehot"] \
        .get_feature_names_out(cat_cols).tolist()
feature_names = feature_names_num + feature_names_cat

X_encoded = preprocess_fitted.transform(X)
X_encoded_dense = X_encoded.toarray() if issparse(X_encoded) else X_encoded

for name, y in y_map.items():
    if not np.issubdtype(y.dtype, np.number):
        print(f"[{name}] target non-numeric -> skipping feature importance.")
        continue

    rf = RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1)
    rf.fit(X_encoded_dense, y)

    imps = pd.Series(rf.feature_importances_, index=feature_names).sort_values(ascending=False)
    topk = imps.head(15)

    print(f"\nTop features for [{name}]:")
    display(topk.to_frame("importance"))

    plt.figure()
    topk.sort_values().plot(kind="barh")
    plt.title(f"Top Feature Importances for {name}")
    plt.xlabel("Importance")
    plt.gca().invert_yaxis()
    plt.show()

# ============================
# 9) Save trained full-data regressors
# ============================
out_dir = "/kaggle/working/models"
os.makedirs(out_dir, exist_ok=True)

for name, y in y_map.items():
    if not np.issubdtype(y.dtype, np.number):
        continue
    pipe_full = Pipeline(steps=[
        ("prep", preprocess),
        ("model", RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1))
    ])
    pipe_full.fit(X, y)
    joblib.dump(pipe_full, os.path.join(out_dir, f"rf_regressor_{name}.joblib"))

print("Saved models to:", out_dir)

# ============================
# 10) Quick EDA prints
# ============================
print("\n=== BASIC EDA ===")
print("Shape:", df.shape)

print("\nNumeric summary:")
display(df.describe(include="number"))

print("\nCategorical summary:")
display(df.describe(include="object"))

# ============================
# 11) SHAP EXPLANATIONS (Sustainability model)
# ============================
import shap

# Refit a pipeline on full data for "sustainability" regression (numeric check)
if np.issubdtype(y_map["sustainability"].dtype, np.number):
    shap_pipe = Pipeline(steps=[("prep", preprocess), ("model", reg_model)])
    shap_pipe.fit(X, y_map["sustainability"])

    # Encoded features
    X_enc = shap_pipe.named_steps["prep"].transform(X)
    X_enc_dense = X_enc.toarray() if issparse(X_enc) else X_enc

    # TreeExplainer for RandomForestRegressor
    explainer = shap.TreeExplainer(shap_pipe.named_steps["model"])
    shap_values = explainer.shap_values(X_enc_dense)

    # Build feature name list again (safe for no-cats)
    feature_names_shap = num_cols[:]
    if len(cat_cols):
        feature_names_shap += preprocess_fitted.named_transformers_["cat"]["onehot"] \
            .get_feature_names_out(cat_cols).tolist()

    # Summary plot
    shap.summary_plot(shap_values, features=X_enc_dense, feature_names=feature_names_shap)

    # Dependence plot (pick a sensible feature that exists)
    # If "att1" doesn't exist, fall back to the first numeric column
    dep_feat = "att1" if "att1" in feature_names_shap else (feature_names_shap[0] if len(feature_names_shap) else None)
    if dep_feat is not None:
        shap.dependence_plot(dep_feat, shap_values, X_enc_dense, feature_names=feature_names_shap)

    # Waterfall / force for one respondent
    sample_idx = 0
    shap.initjs()

    # New SHAP Explanation object (waterfall)
    try:
        exp = shap.Explanation(
            values=shap_values[sample_idx],
            base_values=explainer.expected_value,
            data=X_enc_dense[sample_idx],
            feature_names=feature_names_shap
        )
        shap.plots.waterfall(exp, max_display=15)
    except Exception as e:
        print("Waterfall explanation failed:", e)

    # Matplotlib force plot fallback
    try:
        shap.force_plot(
            base_value=explainer.expected_value,
            shap_values=shap_values[sample_idx, :],
            features=X_enc_dense[sample_idx, :],
            feature_names=feature_names_shap,
            matplotlib=True
        )
    except Exception as e:
        print("Force plot failed:", e)
else:
    print("Sustainability target is non-numeric; skipping SHAP.")
